In [10]:
import pandas as pd
from urllib.request import urlretrieve
import numpy as np

In [3]:
from sentence_transformers import SentenceTransformer, util

In [48]:
pip install ko-sentence-transformers

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 36, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\smhrd\AppData\Local\Temp\pip-install-8zj148vf\ko-sentence-transformers_28e6ce45e4a74361a06b887bba2cd3d7\setup.py", line 6, in <module>
      long_description = f.read()
  UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 30: illegal multibyte sequence
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [21]:
pip install sentence_transformers

     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.3/6.3 MB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
     ------------------------------------- 190.3/190.3 kB 11.3 MB/s eta 0:00:00
     ------------------------------------- 268.0/268.0 kB 16.1 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 11.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b8f21324dc9716ab7a18d4edb349d800da03a2bd59434c16ec1c770f2df16ad1
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\83\71\2b\40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence_tran

### Sbert 사용 하는데 순서

3. 명사 단위 토큰화해서 한번 ㄱ

후보 모델

paraphrase-multilingual-mpnet-base-v2
무난

paraphrase-multilingual-MiniLM-L12-v2
==> 임베딩 단어 vector값 너무 멀어

distiluse-base-multilingual-cased-v1
===> 단어 단위 일치도가 유사도 벡터값 의미 x?


In [64]:
data = pd.read_csv('../시놉시스요약ansi.csv', encoding='cp949')

In [57]:
data = pd.read_csv('../웹툰최종데이터.csv', encoding='euc-kr')

In [ ]:
# paraphrase-multilingual-mpnet-base-v2
# jhgan/ko-sbert-nli 
# jhgan/ko-sbert-sts
# paraphrase-multilingual-MiniLM-L12-v2
model = SentenceTransformer("jhgan/ko-sbert-nli")

In [65]:

embeddings = model.encode(data['synopsis'], convert_to_tensor=True)
embeddings2 = model.encode(test, convert_to_tensor=True)
# embeddings2 = model.encode(data['synopsis'],
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores = cosine_scores.cpu()

In [70]:
top_k = 10
jj=7
#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cosine_scores[jj], range(top_k))[0:top_k]
# 제일 높은 얘의 index 구하는거임. 

print("\n\n======================\n\n")
print("입력 시놉시스 :", data['synopsis'][jj])
print("\nTop 5 젤 비슷한 시놉시스:")

k = 0
for idx in top_results[0:top_k]:
    if k ==0 :
        k+=1
        continue
    print(data['title'][int(idx)]+" :: ",data['synopsis'][int(idx)].strip(),  cosine_scores[jj][int(idx)])
    print("\n")





입력 시놉시스 : 평범한 뱃사공으로 살고 있는 주인공. 하지만 그의 정체는 전설적인 검법의 후계자다. 이제 진시황이 남긴 비서를 향한 살수행이 시작된다.

Top 5 젤 비슷한 시놉시스:
장씨세가 호위무사 ::  은둔 고수. 호위 무사 되다. 무협 시대극 tensor(0.6400)


천화서고 대공자 ::  넉넉한 풍채가 매력적인 무림맹주 후공은 어느 날 느닷없이 웬 비루먹을 청년의 몸에 들어오게 된다.청년의 이름은 범항.강호 3대 서고인  ‘천화서고’의 대공자이자답이 없는 자살희망자.후공은 환혼의 원인을 파악하기 위해범항의 몸을 단련하기 시작하고과거와는 180도 달라진 천화서고 대공자의 움직임에온 강호가 주목하기 시작하는데… tensor(0.6340)


무림서부 ::  황제가 무공을 통한 철혈정치로 제국을 천년동안 유지함에 따라, 오로지 황군 무공만이 힘이요 법도로 살아남고, 고전 무공들은 스러지고 만다. 이런 세계에 환생한 무협지 마니아, 주인공 장건. 과연 그는 동경하던 고전 무공을 되살릴 수 있을까. 서부 대륙 개척지에서 벌어지는 정통 무협의 정수! tensor(0.6229)


사신 ::  형의 복수를 위해 살천문의 무사 황정을 죽인 소년 종리추.살천문의 추격을 피하기 위해 살혼부의 살수 적지인살에게몸을 의탁하게 되면서 십망(十忘)이라는 거대한 소용돌이에 휩쓸린다.십망의 그늘에서 벗어나기 위해선 전설의 경지인 사무령이 되는 수밖에 없다!종리추는 과연 사무령이 되어정파 무림이 만들어놓은 숨막히는 규율 속에서 자유를 찾을 수 있을까? tensor(0.5986)


악당과 악당이 만나면 ::  귀족들을 상대로 악명을 떨치던 제국 뒷세계 조직의 보스. 어느 날 황제로부터 대공을 암살해 달라는 의뢰를 받게 된다. 조직원들과 평범하게 살고 싶었던 보스는 의뢰를 수락하고 대공성에 잠입해 암살 시도하는데…. 하지만 어째서인지 죽지 않아? 한순간에 사랑을 속삭이는 대공에게서 보스는 무사히 탈출할 수 있을까? tensor(0.5970)


망나니 소교주로 환생